# Linear y softmax finales

Por último tenemos dos capas más, una `Linear` y una `Softmax`

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_linear.png" alt="Add and norm" style="width:425px;height:626px;">
  <img src="Imagenes/transformer_architecture_model_softmax.png" alt="Encoder Feed Forward" style="width:425px;height:626px;">
</div>


## Linear

Como hemos dicho el transformer se pensó como un traductor de un idioma a otro, así que ponte que lo queremos usar para hacer un traductor del español al inglés. Como hemos visto hasta ahora, la matriz que sale del decoder es una matriz de tamaño $\left(m_D \times n_D\right)$, pero esto es una matriz de secuencias, nosotros lo que necesitamos es predecir la siguiente palabra. Pues con el módulo `Linear` lo que se hace es contruir un última red fully connected de manera que a la salida tengamos un vector con un montón de posibles tokens del inglés

Y ¿cuántos tokens tenemos que tener en ese vector? Todos los que tengamos, es decir, si hemos definido dos millones de tokens para definir todas las posibles palabras del ingĺes, a la salida deberemos tener un vector con esos dos millones de tokens, a esto normalmente se le llama `vocab size` o tamaño de nuestro vocabulario.

Por lo que la capa `Linear` es una red fully connected que transformará nuestra matriz de tamaño $\left(m_E \times n_E\right)$ a un vector de tamaño `vocab size`. Y lo hará con unos pesos que en función de la matriz que salga del trnasformer, tras todos los mecanismos de atención, haga que tengan más valor unos tokens que otros

## Softmax

Y de todos los posibles tokens de nuestro vocabulario ¿con cuál nos qudamos? Pues para eso tenemos el último módulo `Softmax` con el que convertimos todos los elementos de nuestro vector en probabilidades, de manera que nos quedamos con el token que tenga más probabilidad

## Implementación

Creamos dos clases para estos últimos módulos

In [13]:
import torch.nn as nn

class Linear(nn.Module):
    def __init__(self, dim_embedding, vocab_size):
        super().__init__()
        self.linear = nn.Linear(dim_embedding, vocab_size)
        
    def forward(self, x):
        x = self.linear(x)
        return x

class Softmax(nn.Module):
    def __init__(self):
        super().__init__()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.softmax(x)
        return x

Lo probamos

Como vamos a necesitar la salida del encoder volvemos a escribir todas las clases del encoder

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        """
        Args:
            vocab_size: size of vocabulary
            embed_dim: dimension of embeddings
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            out: embedding vector
        """
        return self.embedding(x)

class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence_len, embedding_model_dim):
        """
        Args:
            seq_len: length of input sequence
            embed_model_dim: demension of embedding
        """
        super().__init__()
        self.embedding_dim = embedding_model_dim

        # create constant 'positional_encoding' matrix with values dependant on pos and i
        positional_encoding = torch.zeros(max_sequence_len, self.embedding_dim)
        for pos in range(max_sequence_len):
            for i in range(0, self.embedding_dim, 2):
                positional_encoding[pos, i]     = math.sin(pos / (10000 ** ((2 *     i) / self.embedding_dim)))
                positional_encoding[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i+1)) / self.embedding_dim)))
        positional_encoding = positional_encoding.unsqueeze(0)
        self.register_buffer('positional_encoding', positional_encoding)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            x: output
        """
        # make embeddings relatively larger
        x = x * math.sqrt(self.embedding_dim)
        
        # add encoding matrix to embedding (x)
        sequence_len = x.size(1)
        # x = x + torch.autograd.Variable(self.positional_encoding[:,:sequence_len], requires_grad=False)
        x = x + self.positional_encoding[:,:sequence_len]
        return x

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, query, key, value, mask=None):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
            mask: mask matrix (optional)
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / math.sqrt(self.dim_embedding)
        # Mask (optional)
        if mask is not None:
            scale = scale.masked_fill(mask == 0, float('-inf'))
        # softmax
        attention_matrix = torch.nn.functional.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads
        
        self.proyection_Q = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_K = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_V = nn.Linear(dim_embedding, dim_embedding)
        self.attention = nn.Linear(dim_embedding, dim_embedding)

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
    
    def forward(self, Q, K, V, mask=None):
        """
        Args:
            Q: query vector
            K: key vector
            V: value vector
            mask: mask matrix (optional)

        Returns:
            output vector from multi-head attention
        """
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        proyection_Q = self.proyection_Q(Q).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, -1, self.heads, self.dim_proyection)
        
        # transpose to get dimensions bs * h * sl * d_model
        proyection_Q = proyection_Q.transpose(1,2)
        proyection_K = proyection_K.transpose(1,2)
        proyection_V = proyection_V.transpose(1,2)

        # calculate attention
        scaled_dot_product_attention = self.scaled_dot_product_attention(proyection_Q, proyection_K, proyection_V, mask=mask)
        
        # concatenate heads and put through final linear layer
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)
        
        output = self.attention(concat)
    
        return output

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding (int): Embedding dimension.
        """
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        """
        Args:
            x (torch.Tensor): Input tensor.
            sublayer (torch.Tensor): Sublayer tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.normalization(torch.add(x, sublayer))

class FeedForward(nn.Module):
    def __init__(self, dim_embedding, increment=4):
        super().__init__()
        self.feed_forward = nn.Sequential(
            nn.Linear(dim_embedding, dim_embedding*increment),
            nn.ReLU(),
            nn.Linear(dim_embedding*increment, dim_embedding)
        )
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        x = self.feed_forward(x)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, heads, dim_embedding):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.add_and_norm_1 = AddAndNorm(dim_embedding)
        self.feed_forward = FeedForward(dim_embedding)
        self.add_and_norm_2 = AddAndNorm(dim_embedding)
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        multi_head_attention = self.multi_head_attention(x, x, x)
        add_and_norm_1 = self.add_and_norm_1(x, multi_head_attention)
        feed_forward = self.feed_forward(add_and_norm_1)
        add_and_norm_2 = self.add_and_norm_2(add_and_norm_1, feed_forward)
        return add_and_norm_2

class Encoder(nn.Module):
    def __init__(self, heads, dim_embedding, Nx):
        super().__init__()
        self.encoder_layers = nn.ModuleList([EncoderLayer(heads, dim_embedding) for _ in range(Nx)])
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x)
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, dim_embedding, max_sequence_len, heads, Nx):
        super().__init__()
        self.input_embedding = Embedding(vocab_size, dim_embedding)
        self.positional_encoding = PositionalEncoding(max_sequence_len, dim_embedding)
        self.encoder = Encoder(heads, dim_embedding, Nx)
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        input_embedding = self.input_embedding(x)
        positional_encoding = self.positional_encoding(input_embedding)
        encoder = self.encoder(positional_encoding)
        return encoder


Volvemos a definir la función que obtiene el embbeding más el positional encoding de BERT

In [3]:
import torch
from transformers import BertModel, BertTokenizer

def extract_embeddings(input_sentences, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    # tokenización de lote
    inputs = tokenizer(input_sentences, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    token_embeddings = outputs[0]
    
    # Los embeddings posicionales están en la segunda capa de los embeddings de la arquitectura BERT
    positional_encodings = model.embeddings.position_embeddings.weight[:token_embeddings.shape[1], :].detach().unsqueeze(0).repeat(token_embeddings.shape[0], 1, 1)

    embeddings_with_positional_encoding = token_embeddings + positional_encodings

    # convierte las IDs de los tokens a tokens
    tokens = [tokenizer.convert_ids_to_tokens(input_id) for input_id in inputs['input_ids']]

    return tokens, inputs['input_ids'], token_embeddings, positional_encodings, embeddings_with_positional_encoding

Creamos una sentencia para el encoder, ya que ahora va a entrar una sentencia al encoder y otra al decoder

In [4]:
sentence_encoder = "I gave the dog a bone because it was hungry"
tokens_encoder, input_ids_encoder, token_embeddings_encoder, positional_encodings_encoder, embeddings_with_positional_encoding_encoder = extract_embeddings(sentence_encoder)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Creamos un objeto `encoder` y obtenemos su salida

In [5]:
vocab_size = 30522
dim_embedding = token_embeddings_encoder.shape[-1]
max_sequence_len = token_embeddings_encoder.shape[1]
heads = 8
Nx = 6
transformer_encoder = TransformerEncoder(vocab_size, dim_embedding, max_sequence_len, heads, Nx)
encoder_output = transformer_encoder(input_ids_encoder)
encoder_output.shape

torch.Size([1, 12, 768])

Ahora generamos una sentencia para el decoder, lo que haremos será generar la sentencia del encoder traducida al español

In [6]:
sentence_encoder = "I gave the dog a bone because it was hungry"
sentence_decoder = "Le di un hueso al perro porque tenía hambre"
tokens_decoder, input_ids_decoder, token_embeddings_decoder, positional_encodings_decoder, embeddings_with_positional_encoding_decoder = extract_embeddings(sentence_decoder)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Instanciamos las clases del decoder y generamos su salida

In [7]:
def create_mask(sequence_len):
    """
    Args:
        sequence_len: length of sequence
        
    Returns:
        mask matrix
    """
    mask = torch.tril(torch.ones((sequence_len, sequence_len)))
    return mask
sequence_len = input_ids_decoder.shape[1]
mask = create_mask(sequence_len)

dim_embedding = embeddings_with_positional_encoding_decoder.shape[-1]
heads = 8
masked_multi_head_attention = MultiHeadAttention(heads=heads, dim_embedding=dim_embedding)
add_and_norm_3 = AddAndNorm(dim_embedding)
encoder_decoder_multi_head_attention = MultiHeadAttention(heads=heads, dim_embedding=dim_embedding)
add_and_norm_4 = AddAndNorm(dim_embedding)
feed_forward = FeedForward(dim_embedding)
add_and_norm_5 = AddAndNorm(dim_embedding)

K = embeddings_with_positional_encoding_decoder
V = embeddings_with_positional_encoding_decoder
Q = embeddings_with_positional_encoding_decoder
masked_attention = masked_multi_head_attention(Q, K, V, mask=mask)
masked_attention_add_and_norm = add_and_norm_3(embeddings_with_positional_encoding_decoder, masked_attention)

K = encoder_output
V = encoder_output
Q = masked_attention_add_and_norm
encoder_decoder_attention = encoder_decoder_multi_head_attention(Q, K, V)
encoder_decoder_attention_add_and_norm = add_and_norm_4(masked_attention_add_and_norm, encoder_decoder_attention)

feed_forward_output = feed_forward(encoder_decoder_attention_add_and_norm)
add_and_norm_5_output = add_and_norm_5(encoder_decoder_attention_add_and_norm, feed_forward_output)
add_and_norm_5_output.shape

torch.Size([1, 16, 768])

Creamos ahora un objeto de la clase `Linear` y calculamos su salida

In [14]:
linear = Linear(dim_embedding, vocab_size)
linear_output = linear(add_and_norm_5_output)
linear_output.shape

torch.Size([1, 16, 30522])

Como vemos ahora obtenemos una salida de 1x16x30522, es decir 1 batch, 

In [9]:
softmax = Softmax()
softmax_output = softmax(linear_output)
softmax_output.shape

torch.Size([1, 16, 30522])